In [14]:
from pathlib import Path
import pandas as pd
import os
from dotenv import load_dotenv

DOTENV_PATH = (Path.cwd().parent / ".env").resolve()
print("looking for:", DOTENV_PATH)
loaded = load_dotenv(dotenv_path=DOTENV_PATH)

EXCEL_PATH = Path("daily_report.xlsx")
SHEET_NAME = "reports"
print("looking for:", EXCEL_PATH)

df = pd.read_excel(EXCEL_PATH, sheet_name=SHEET_NAME)

required = {"date","trainee","did","learned","improve"}
lack = required - set(df.columns)

df.head(3)


looking for: /Users/atanoue/Downloads/llmdev/.env
looking for: daily_report.xlsx


,date,trainee,did,learned,improve
0,2025-09-22,田之上明美,Lesson12まで完了、Lesson 13の課題を検討中,OpenAI APIのツール呼出のやり方を学べた、Web検索連動チャットボットのソースコード...,とても難しくなってきたので、一つ一つ理解をしていくのに骨が折れる。課題にだいぶ時間がかかって...


In [15]:
import re
from openai import OpenAI
client = OpenAI(api_key=os.getenv("API_KEY"))

MODEL = "gpt-4o-mini"

def build_prompt(row: dict) -> str:
  return f"""
あなたは研修日報の要約アシスタントです。入力から以下3点を日本語で簡潔に出力してください。
- 今日の要点（didを要約）
- 課題（learnedから、今見えてきた課題を抽出）
- 次のアクション（improveを要約し、行動の形に）

制約:
- 箇条書きにせず、各項目は1〜2文で。
- 主語は省略してOK。事実ベースで簡潔に。
- 固有名詞は入力にない限り推測しない。
- 出力は次のラベル順で1回ずつ:
  今日の要点：...
  課題：...
  次のアクション：...

【入力】
実施したこと(did):
{row['did']}

気づき/学んだこと(learned):
{row['learned']}

改善/トライすること(improve):
{row['improve']}
"""

def call_llm(prompt: str) -> dict:
  messages = [
    {"role":"system","content":"You are a concise Japanese business writing assistant."},
    {"role":"user","content":prompt},
  ]
  resp = client.chat.completions.create(
    model=MODEL,
    messages=messages,
    temperature=0.2,
  )
  text = resp.choices[0].message.content.strip()

  def ext(label, fb=""):
    m = re.search(rf"{label}\s*[：:]\s*(.+)", text)
    return (m.group(1).strip() if m else fb)

  key_points   = ext("今日の要点")
  issues       = ext("課題")
  next_actions = ext("次のアクション")

  if not (key_points and issues and next_actions):
    # ラベル崩れの保険をかけておく
    parts = [p.strip(" ・-　") for p in re.split(r"[\n]+", text) if p.strip()]
    if len(parts) >= 3:
      key_points, issues, next_actions = parts[:3]
    else:
      key_points   = key_points   or text
      issues       = issues       or "（課題の抽出に失敗）"
      next_actions = next_actions or "（次のアクションの抽出に失敗）"

  return {
    "key_points": key_points,
    "issues": issues,
    "next_actions": next_actions
  }


In [ ]:
def make_slack_text(row: dict, s: dict) -> str:
  return (
    f"お疲れ様です。{row.get('trainee','')} です。\n"
    f"・実施したこと\n"
    f"　・{s['key_points']}\n"
    f"・気づき/学んだこと\n"
    f"　・{s['issues']}\n"
    f"・改善/トライすること\n"
    f"　・{s['next_actions']}"
  )


In [20]:
from datetime import datetime

generated = []
for idx, r in df.iterrows():
  row = r.to_dict()
  prompt = build_prompt(row)
  s = call_llm(prompt)           #dictが返る
  text = make_slack_text(row, s)
  generated.append(text)

# 見やすく区切って出力させる
for i, t in enumerate(generated, start=1):
  print(t)
  print()

お疲れ様です。田之上明美 です。）
・実施したこと
　・Lesson12まで完了し、現在Lesson 13の課題を検討中。OpenAI APIのツール呼出やWeb検索連動チャットボットのソースコード作成、LLMを活用したExcel操作の自動化、スクレイピングの技術を習得した。
・気づき/学んだこと
　・課題の理解が難しくなっており、一つ一つの内容を把握するのに時間がかかっている。
・改善/トライすること
　・各課題を分解し、理解を深めるために時間をかけて取り組む。

